# Dependencies

In [ ]:
devtools::install_github("aertslab/AUCell")
devtools::install_github("aertslab/RcisTarget")

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install('GenomicRanges')

In [ ]:
devtools::install_github("aertslab/cisTopic")

In [ ]:
BiocManager::install(c('Rsubread', 'umap', 'Rtsne', 'ComplexHeatmap', 'fastcluster', 'data.table', 'rGREAT', 'ChIPseeker', 'TxDb.Hsapiens.UCSC.hg19.knownGene', 'org.Hs.eg.db'))

# Model Building 
### *(load precomputed obj)*

In [ ]:
cisTopicObject <- readRDS('Melanoma_data/Melanoma_tutorial/cisTopicObject_melanoma_warpLDA.Rds')

# Graphics

In [ ]:
# UMAP
cisTopicObject <- runUmap(cisTopicObject, target='cell')

par(mfrow=c(1,2))
plotFeatures(cisTopicObject, method='Umap', target='cell', topic_contr=NULL, colorBy=c('cellLine', 'LineType'), cex.legend = 0.8, factor.max=.75, dim=2, legend=TRUE, col.low='darkgreen', col.mid='yellow', col.high='brown1', intervals=20)

par(mfrow=c(2,5))
plotFeatures(cisTopicObject, method='Umap', target='cell', topic_contr='Probability', colorBy=NULL, cex.legend = 0.8, factor.max=.75, dim=2, legend=TRUE)

In [ ]:
# predictive distributions == likelihood of reagion in each cell
# predictive distributions == can be used (with AUCell) for estimating the enrichment of epigenomic signatures

pred.matrix <- predictiveDistribution(cisTopicObject)

# Obtain signatures
path_to_signatures <- '/data/ChIP-seq_signatures/'
ChIP_Seq_signatures <- paste(path_to_signatures, list.files(path_to_signatures), sep='')
labels  <- c('MITF', 'SOX10', 'TFAP2A')
cisTopicObject <- getSignaturesRegions(cisTopicObject, ChIP_Seq_signatures, labels=labels, minOverlap = 0.4)

# Compute cell rankings
library(AUCell)
aucellRankings <- AUCell_buildRankings(pred.matrix, plot=FALSE, verbose=FALSE)

# Check signature enrichment in cells
cisTopicObject <- signatureCellEnrichment(cisTopicObject, aucellRankings, selected.signatures='all', aucMaxRank = 0.1*nrow(aucellRankings), plot=FALSE)

# Plot
par(mfrow=c(2,2))
plotFeatures(cisTopicObject, method='Umap', target='cell', topic_contr=NULL, colorBy=c('LineType', 'MITF', 'SOX10', 'TFAP2A'), cex.legend = 0.8, factor.max=.75, dim=2, legend=TRUE, intervals=20)